In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()
import yfinance as yf
from importlib import reload

pd.set_option('max_colwidth', 400)

In [77]:
import scripts.preprocess_stocks
reload(scripts.preprocess_stocks)
from scripts.preprocess_stocks import preprocess_stock_data
stocks = pd.read_excel("trading_data/congress-trading-all.xlsx", parse_dates=["Traded", "Filed", "Quiver_Upload_Time"])
stocks = preprocess_stock_data(stocks)
stocks.describe(include="all")

,Ticker,TickerType,Company,Traded,Transaction,Subholding,Description,Name,Filed,Party,District,Chamber,Comments,State,Min_Trade_Size,Max_Trade_Size
count,46172,33236,46171,46172,46172,27183,4230,46172,46172,46172,36132,46172,811,46172,4.617200e+04,4.617200e+04
unique,3844,6,6381,NaN,5,518,2283,292,NaN,3,196,2,194,50,NaN,NaN
top,MSFT,ST,MICROSOFT CORPORATION,NaN,Purchase,THOMAS C MACARTHUR AND DEBORAH A MACARTHUR,DIVIDEND REINVESTMENT,Thomas Macarthur,NaN,R,NJ03,House,R,New Jersey,NaN,NaN
freq,764,24087,634,NaN,22829,1841,255,2758,NaN,24367,2758,36132,184,6041,NaN,NaN
mean,NaN,NaN,NaN,2019-03-03 22:36:56.824049152,NaN,NaN,NaN,NaN,2019-05-06 08:22:22.320020992,NaN,NaN,NaN,NaN,NaN,1.808105e+04,6.269328e+04
min,NaN,NaN,NaN,2012-02-27 00:00:00,NaN,NaN,NaN,NaN,2014-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,1.000000e-02,0.000000e+00
25%,NaN,NaN,NaN,2016-12-30 00:00:00,NaN,NaN,NaN,NaN,2017-02-03 00:00:00,NaN,NaN,NaN,NaN,NaN,1.001000e+03,1.500000e+04
50%,NaN,NaN,NaN,2019-05-28 00:00:00,NaN,NaN,NaN,NaN,2019-09-02 00:00:00,NaN,NaN,NaN,NaN,NaN,1.001000e+03,1.500000e+04
75%,NaN,NaN,NaN,2021-02-17 00:00:00,NaN,NaN,NaN,NaN,2021-06-14 00:00:00,NaN,NaN,NaN,NaN,NaN,1.500100e+04,5.000000e+04
max,NaN,NaN,NaN,2024-02-21 00:00:00,NaN,NaN,NaN,NaN,2024-02-23 00:00:00,NaN,NaN,NaN,NaN,NaN,2.500000e+07,5.000000e+07


In [78]:
symbol2sector = pd.read_csv('trading_data/symbol2sector.csv')
symbol2sector.head()

,Ticker,Sector,Industry,YFQuoteType
0,CELG,Healthcare,Drug Manufacturers - General,EQUITY
1,DISCA,Communication Services,Entertainment,EQUITY
2,DISCK,Communication Services,Entertainment,EQUITY
3,FDC,Technology,Information Technology Services,EQUITY
4,AGN,Healthcare,Drug Manufacturers - General,EQUITY


In [79]:
stocks = pd.merge(stocks, symbol2sector, on="Ticker", how="left")
stocks.head()

,Ticker,TickerType,Company,Traded,Transaction,Subholding,Description,Name,Filed,Party,District,Chamber,Comments,State,Min_Trade_Size,Max_Trade_Size,Sector,Industry,YFQuoteType
0,PANW,OP,"PALO ALTO NETWORKS, INC.",2024-02-21,Purchase,NaN,PURCHASED 20 CALL OPTIONS WITH A STRIKE PRICE OF $200 AND AN EXPIRATION DATE OF 1/17/25.,Nancy Pelosi,2024-02-23,D,CA11,House,NaN,California,100001.0,250000.0,Technology,Software - Infrastructure,EQUITY
1,PANW,OP,"PALO ALTO NETWORKS, INC.",2024-02-12,Purchase,NaN,PURCHASED 50 CALL OPTIONS WITH A STRIKE PRICE OF $200 AND AN EXPIRATION DATE OF 1/17/25.,Nancy Pelosi,2024-02-23,D,CA11,House,NaN,California,500001.0,1000000.0,Technology,Software - Infrastructure,EQUITY
2,SPG,ST,"SIMON PROPERTY GROUP, INC. COMMON STOCK",2024-02-02,Purchase,CHARLES SCHWAB MINOR ROTH IRA,THIS TRANSACTION WAS MADE BY LILIANNA SPARTZ'S MINOR ROTH IRA.,Victoria Spartz,2024-02-15,R,IN05,House,NaN,Indiana,1001.0,15000.0,Real Estate,REIT - Retail,EQUITY
3,SPG,ST,"SIMON PROPERTY GROUP, INC. COMMON STOCK",2024-02-02,Purchase,CHARLES SCHWAB ROTH IRA,THIS TRANSACTION WAS MADE BY JASON SPARTZ'S ROTH IRA.,Victoria Spartz,2024-02-15,R,IN05,House,NaN,Indiana,50001.0,100000.0,Real Estate,REIT - Retail,EQUITY
4,SPG,ST,"SIMON PROPERTY GROUP, INC. COMMON STOCK",2024-02-02,Purchase,CHARLES SCHWAB MINOR ROTH IRA,THIS TRANSACTION WAS MADE BY INGRID SPARTZ'S MINOR ROTH IRA.,Victoria Spartz,2024-02-15,R,IN05,House,NaN,Indiana,1001.0,15000.0,Real Estate,REIT - Retail,EQUITY


In [80]:
stocks[stocks.YFQuoteType == "NONE"].Ticker.value_counts()

Ticker
ABB      19
CDK      18
DEACU    18
WGP      18
CTXS     18
         ..
FTSI      1
HIFR      1
JCAP      1
VER       1
NTRI      1
Name: count, Length: 449, dtype: int64

In [76]:
stocks[stocks.Ticker == "ABB"]

,Ticker,TickerType,Company,Traded,Transaction,Subholding,Description,Name,Filed,Party,District,Chamber,Comments,State,Min_Trade_Size,Max_Trade_Size,Sector,Industry,YFQuoteType
2815,ABB,ST,ABB LTD,2023-05-11,Purchase,HERN FAMILY REVOCABLE TRUST > BROKERAGE INVESTMENT ACCOUNT,DIVIDEND REINVESTMENT,Kevin Hern,2023-06-13,R,OK01,House,NaN,Oklahoma,1001.0,15000.0,Not Found,Not Found,NONE
6503,ABB,ST,ABB LTD,2022-05-11,Purchase,HERN FAMILY REVOCABLE TRUST > BROKERAGE INVESTMENT ACCOUNT,DIVIDEND REINVESTMENT,Kevin Hern,2022-06-14,R,OK01,House,NaN,Oklahoma,1001.0,15000.0,Not Found,Not Found,NONE
7340,ABB,ST,ABB LTD,2022-02-17,Sale,NATIONAL FINANCIAL SERVICES/FIDELITY - PREMIERE SELECT IRA INT'L,NaN,John Rutherford,2022-03-16,R,FL04,House,NaN,Florida,1001.0,15000.0,Not Found,Not Found,NONE
8206,ABB,ST,ABB LTD,2021-11-26,Purchase,HERN FAMILY REVOCABLE TRUST > BROKERAGE INVESTMENT ACCOUNT,NaN,Kevin Hern,2021-12-09,R,OK01,House,NaN,Oklahoma,50001.0,100000.0,Not Found,Not Found,NONE
9386,ABB,ST,ABB LTD,2021-07-27,Sale,AMERIPRISE TRADITIONAL IRA,NaN,Katherine M. Clark,2021-08-15,D,MA05,House,NaN,Massachusetts,1001.0,15000.0,Not Found,Not Found,NONE
9388,ABB,ST,ABB LTD,2021-07-27,Sale,AMERIPRISE SEP IRA,NaN,Katherine M. Clark,2021-08-15,D,MA05,House,NaN,Massachusetts,1001.0,15000.0,Not Found,Not Found,NONE
12190,ABB,ST,ABB LTD,2021-01-20,Sale,NATIONAL FINANCIAL SERVICES/FIDELITY - PREMIERE SELECT IRA INT'L,NaN,John Rutherford,2021-02-02,R,FL04,House,NaN,Florida,1001.0,15000.0,Not Found,Not Found,NONE
13420,ABB,ST,ABB LTD,2020-11-10,Purchase,AMERIPRISE SEP IRA,NaN,Katherine M. Clark,2020-12-28,D,MA05,House,NaN,Massachusetts,1001.0,15000.0,Not Found,Not Found,NONE
13533,ABB,ST,ABB LTD,2020-11-05,Purchase,AMERIPRISE TRADITIONAL IRA,NaN,Katherine M. Clark,2020-12-28,D,MA05,House,NaN,Massachusetts,1001.0,15000.0,Not Found,Not Found,NONE
17308,ABB,ST,ABB LTD,2020-04-03,Sale,AMERIPRISE SEP IRA,NaN,Katherine M. Clark,2020-05-29,D,MA05,House,NaN,Massachusetts,1001.0,15000.0,Not Found,Not Found,NONE


In [72]:
yf.Ticker("BFH").info

{'address1': '3095 Loyalty Circle',
 'city': 'Columbus',
 'state': 'OH',
 'zip': '43219',
 'country': 'United States',
 'phone': '614 729 4000',
 'website': 'https://www.breadfinancial.com',
 'industry': 'Credit Services',
 'industryKey': 'credit-services',
 'industryDisp': 'Credit Services',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Bread Financial Holdings, Inc. provides tech-forward payment and lending solutions to customers and consumer-based industries in North America. It offers credit card and other loans financing services, including risk management solutions, account origination, and funding services for private label and co-brand credit card programs, as well as through Bread partnerships; and Comenity-branded general purpose cash-back credit. The company also manages and services the loans it originates for private label, co-brand, and general-purpose credit card programs, and installmen

In [124]:
yf.Ticker("META").info

{'address1': '1 Meta Way',
 'city': 'Menlo Park',
 'state': 'CA',
 'zip': '94025',
 'country': 'United States',
 'phone': '650 543 4800',
 'website': 'https://investor.fb.com',
 'industry': 'Internet Content & Information',
 'industryKey': 'internet-content-information',
 'industryDisp': 'Internet Content & Information',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, and wearables worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment offers Facebook, which enables people to share, discuss, discover, and connect with interests; Instagram, a community for sharing photos, videos, and private messages, as well as feed, stories, reels, video, live, and shops; M

In [16]:
sector2lobbyingcode = pd.read_csv("trading_data/sector2lobbyingcode.csv")
sector2lobbyingcode.head()

,Industry,Category1,Category2,Category3
0,Advertising Agencies,ADV,NaN,NaN
1,Aerospace & Defense,AER,DEF,NaN
2,Agricultural Inputs,AGR,NaN,NaN
3,Airlines,AER,TRA,TOU
4,Airports & Air Services,AER,TRA,TOU


In [17]:
# Our script found the sector iff it found the industry
symbol2sector[(symbol2sector.Industry == 'Not Found') ^ (symbol2sector.Sector == 'Not Found')]

,Ticker,Sector,Industry,YFQuoteType


In [114]:
stocks[stocks.Industry == "Not Found"].Ticker.value_counts()

Ticker
BRK      202
SPY       85
TWTR      74
FEYE      73
AGN       73
        ... 
CCH        1
TANN       1
GOOCV      1
IGIB       1
GMXRQ      1
Name: count, Length: 1411, dtype: int64

In [19]:
stocks[stocks.Industry == "Not Found"].YFQuoteType.value_counts()

YFQuoteType
NONE           3273
ETF            2040
MUTUALFUND     1813
Not Found       786
EQUITY          367
MONEYMARKET       2
INDEX             2
ECNQUOTE          1
Name: count, dtype: int64

In [27]:
stocks[stocks.YFQuoteType == "NONE"].Industry.value_counts()

Industry
Not Found    3273
Name: count, dtype: int64

In [29]:
stocks[(stocks.YFQuoteType == "EQUITY") & (stocks.Industry == "Not Found")].Ticker.value_counts()

Ticker
BRK.B      194
FISV        43
RDS.A       32
SIVB        17
LUKOY       16
LDP         10
BRK.A        8
PDCE         7
MGDDY        5
NSR          4
LGCY         3
HEI.A        3
APPAD        3
MOG.A        3
HZNP         3
DWTI         2
TRIT         2
VRAY         2
VODPD        1
ZU           1
SBUX.SW      1
AIO          1
AEX.MU       1
SYY.SG       1
BMEZ         1
GSS          1
TACO         1
GMXRQ        1
Name: count, dtype: int64

In [47]:
def normalize_ticker_symbol(symbol):
    if "$" in symbol:
        symbol = symbol[:symbol.find("$")]
    if "." in symbol:
        symbol = symbol[:symbol.find(".")]
    return symbol

normalize_ticker_symbol("BRK.B")
        

'BRK'

In [48]:
stocks.Ticker.apply(normalize_ticker_symbol)

0        PANW
1        PANW
2         SPG
3         SPG
4         SPG
         ... 
46168     CAT
46169      PG
46170     KSU
46171      EP
46172      EP
Name: Ticker, Length: 46173, dtype: object

In [44]:
stocks[stocks.Ticker.str.contains("$", regex=False)].groupby("Ticker").agg("first")

,TickerType,Company,Traded,Transaction,Subholding,Description,Name,Filed,Party,District,Chamber,Comments,State,Min_Trade_Size,Max_Trade_Size,Sector,Industry,YFQuoteType
Ticker,,,,,,,,,,,,,,,,,,
AAIC$B,ST,ARLINGTON ASSET INVESTMENT CORP 7.00%,2023-11-28,Purchase,None,None,Virginia Foxx,2023-12-05,R,NC05,House,None,North Carolina,1001.0,15000.0,Not Found,Not Found,Not Found
AAM$A,ST,"APOLLO ASSET MANAGEMENT, INC. 6.375% SERIES A PREFERRED STOCK",2023-09-22,Sale,None,None,Virginia Foxx,2023-10-06,R,NC05,House,None,North Carolina,15001.0,50000.0,Not Found,Not Found,Not Found
BANC$E,ST,"BANC OF CALIFORNIA, INC. DEPOSITARY SHARES EACH REPRESENTING A 1/40TH INTEREST IN A SHARE OF 7.000% NON- CUMULATIVE PERPETUAL PREFERRED STOCK, SERIES E",2022-03-15,Exchange,SEP IRA,None,Russ Fulcher,2023-05-12,R,ID01,House,None,Idaho,1001.0,15000.0,Not Found,Not Found,Not Found
CLNS$A,None,"COLONY NORTHSTAR, INC. 8.75% SERIES A CUMULATIVE REDEEMABLE PERPETUAL PREFERRED STOCK",2017-05-23,Sale,OPPENHEIMER IRA,None,Dwight Evans,2017-07-07,D,PA02,House,None,Pennsylvania,15001.0,50000.0,Not Found,Not Found,Not Found
CPE$A,None,CALLON PETROLEUM COMPANY PREFERRED SHARES SERIES A 10%,2014-06-02,Sale,None,None,Michael K. Simpson,2014-07-10,R,ID02,House,None,Idaho,15001.0,50000.0,Not Found,Not Found,Not Found
DUK$A,ST,"DUKE ENERGY CORPORATION DEPOSITARY SHARES, EACH REPRESENTING A 1/1,000TH INTEREST IN A SHARE OF 5.75% SERIES A CUMULATIVE REDEEMABLE PERPETUAL PREFERRED STOCK",2024-01-16,Purchase,None,None,Virginia Foxx,2024-02-05,R,NC05,House,None,North Carolina,1001.0,15000.0,Not Found,Not Found,Not Found
EFC$D,ST,ELLINGTON FINANCIAL INC. 7.00% SERIES D CUMULATIVE PERPETUAL REDEEMABLE PREFERRED STOCK,2023-12-14,Exchange,None,ARLINGTON ASSET MANAGEMENT 7% PREFERRED STOCK EXCHANGED FOR ELLINGTON FINANCIAL 7% PREFERRED STOCK,Virginia Foxx,2024-01-04,R,NC05,House,None,North Carolina,15001.0,50000.0,Not Found,Not Found,Not Found
EP$C,None,EL PASO CORPORATION PREFERRED STOCK,2012-03-20,Sale,None,None,Alan S. Lowenthal,2014-05-15,D,CA47,House,None,California,1001.0,15000.0,Not Found,Not Found,Not Found
MS$P,ST,"MORGAN STANLEY DEPOSITARY SHARES, EACH REPRESENTING 1/1,000TH OF A SHARE OF 6.500% NON-CUMULATIVE PREFERRED STOCK, SERIES P",2022-08-10,Purchase,STEPHENS ADVANTAGE ACCOUNT,None,Steve Cohen,2022-08-15,D,TN09,House,None,Tennessee,50001.0,100000.0,Not Found,Not Found,Not Found


In [35]:
yf.Ticker("SBUX").info

{'address1': '2401 Utah Avenue South',
 'city': 'Seattle',
 'state': 'WA',
 'zip': '98134',
 'country': 'United States',
 'phone': '206 447 1575',
 'website': 'https://www.starbucks.com',
 'industry': 'Restaurants',
 'industryKey': 'restaurants',
 'industryDisp': 'Restaurants',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': "Starbucks Corporation, together with its subsidiaries, operates as a roaster, marketer, and retailer of coffee worldwide. The company operates through three segments: North America, International, and Channel Development. Its stores offer coffee and tea beverages, roasted whole beans and ground coffees, single serve products, and ready-to-drink beverages; and various food products, such as pastries, breakfast sandwiches, and lunch items. The company also licenses its trademarks through licensed stores, and grocery and foodservice accounts. The company offers its products under the Starb

In [80]:
symbol2sector[symbol2sector.Ticker.str.startswith("DISC")]

,Ticker,sector,industry
298,WBD,Communication Services,Entertainment


In [84]:
stocks[stocks.Ticker == 'SPY']

,Ticker,TickerType,Company,Traded,Transaction,Subholding,Description,Name,Filed,Party,District,Chamber,Comments,State,Min_Trade_Size,Max_Trade_Size,sector,industry
171,SPY,ST,SPDR S&P 500,2024-01-19,Purchase,IRON GATE GA BROKERAGE PORTFOLIO > IGGA IRA (BM),NaN,Blake Moore,2024-01-24,R,UT01,House,NaN,Utah,100001.0,250000.0,Not Found,Not Found
172,SPY,ST,SPDR S&P 500,2024-01-19,Purchase,IRON GATE GA BROKERAGE PORTFOLIO > IGGA ROTH IRA (JM),NaN,Blake Moore,2024-01-24,R,UT01,House,NaN,Utah,15001.0,50000.0,Not Found,Not Found
199,SPY,ST,SPDR S&P 500,2024-01-19,Purchase,IRON GATE GA BROKERAGE PORTFOLIO > IGGA ROTH IRA (BM),NaN,Blake Moore,2024-01-24,R,UT01,House,NaN,Utah,15001.0,50000.0,Not Found,Not Found
1799,SPY,OP,SPDR S&P 500,2023-08-04,Sale,EQUITABLE RETIREMENT ACCOUNT,SELL OF JAN 24 SPY 440 PUT,Mark Dr Green,2023-08-08,R,TN07,House,NaN,Tennessee,15001.0,50000.0,Not Found,Not Found
2092,SPY,OP,SPDR S&P 500,2023-07-11,Purchase,EQUITABLE RETIREMENT ACCOUNT,13 SPY PUTS STRIKE PRICE $440 EXPIRE 1/19/2024,Mark Dr Green,2023-08-08,R,TN07,House,NaN,Tennessee,15001.0,50000.0,Not Found,Not Found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41472,SPY,Stock,SPDR S&P 500 ETF Trust (NYSEArca),2015-03-25,Sale (Full),NaN,NaN,David Perdue,2015-06-04,R,NaN,Senate,NaN,Georgia,15001.0,50000.0,Not Found,Not Found
42177,SPY,Stock,SPDR S&P 500 ETF (NYSEArca),2015-01-30,Sale (Full),NaN,NaN,Pat Toomey,2015-03-23,R,NaN,Senate,NaN,Pennsylvania,15001.0,50000.0,Not Found,Not Found
42212,SPY,Other Securities,SPDR S&P 500 ETF,2015-01-28,Sale (Partial),NaN,NaN,Dan Sullivan,2016-08-03,R,NaN,Senate,NaN,Alaska,1001.0,15000.0,Not Found,Not Found
43588,SPY,NaN,SPDR S&P 500,2014-07-31,Purchase,LB,REINVESTED DIVIDENDS,Richard L. Hanna,2014-08-13,R,NY22,House,NaN,New York,15001.0,50000.0,Not Found,Not Found


In [85]:
ticker = yf.Ticker("SPY")
ticker.info

{'longBusinessSummary': 'The Trust seeks to achieve its investment objective by holding a portfolio of the common stocks that are included in the index (the “Portfolio”), with the weight of each stock in the Portfolio substantially corresponding to the weight of such stock in the index.',
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 523.07,
 'open': 523.83,
 'dayLow': 520.97,
 'dayHigh': 524.38,
 'regularMarketPreviousClose': 523.07,
 'regularMarketOpen': 523.83,
 'regularMarketDayLow': 520.97,
 'regularMarketDayHigh': 524.38,
 'trailingPE': 26.303535,
 'volume': 61184748,
 'regularMarketVolume': 61184748,
 'averageVolume': 74874477,
 'averageVolume10days': 75937860,
 'averageDailyVolume10Day': 75937860,
 'bid': 521.64,
 'ask': 521.68,
 'bidSize': 800,
 'askSize': 1200,
 'yield': 0.012999999,
 'totalAssets': 503321591808,
 'fiftyTwoWeekLow': 403.74,
 'fiftyTwoWeekHigh': 524.61,
 'fiftyDayAverage': 503.3446,
 'twoHundredDayAverage': 460.6849,
 'trailingAnnualDividendRate': 5.662

In [86]:
len(stocks.Ticker.unique())

3882

In [40]:
stocks[stocks.Ticker == "NVDA"].Name.value_counts()

Name
Josh Gottheimer              32
Roberts, Pat                 25
Thomas Suozzi                21
Kathy Manning                18
Wyden, Ron                   13
Gilbert Cisneros             10
Whitehouse, Sheldon          10
Pete Sessions                10
Donald Sternoff Beyer Jr      9
Daniel Meuser                 8
Nancy Pelosi                  7
Blake Moore                   6
Tuberville, Tommy             6
Carper, Thomas R.             5
John Curtis                   4
Donald Sternoff  Beyer Jr     4
Nicholas Van Taylor           3
Mullin, Markwayne             3
Alan S. Lowenthal             3
Sullivan, Dan                 3
Morgan Mcgarvey               3
Roger W. Marshall             3
Dan Newhouse                  3
Brenda Lulenar Lawrence       2
Loeffler, Kelly               2
David Cheston Rouzer          2
Daniel Goldman                2
Dean Phillips                 2
HON. GREG STANTON             2
Debbie Dingell                2
James R. Langevin             1
Mik